2602160750 - Bryan Orville Audric

In [10]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


Dataset : https://www.kaggle.com/code/mehakiftikhar/amazon-sales-dataset-eda/input


In [11]:
Use_Columns = ['product_id','user_id','rating','product_name']
Amazon_Sales = pd.read_csv("/content/drive/MyDrive/Dataset/amazon.csv",usecols=Use_Columns)

In [27]:
#Convert rating datatype from obj to numeric(float,int,etc)
Amazon_Sales['rating'] = pd.to_numeric(Amazon_Sales['rating'], errors='coerce')

Amazon_Sales.dropna(subset=['rating'], inplace=True)
Amazon_Sales.head()

,product_id,product_name,rating,user_id
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,4.2,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB..."
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,4.0,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX..."
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,3.9,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ..."
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,4.2,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S..."
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,4.2,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH..."


In [13]:
print("Number of Rating:",len(Amazon_Sales))
print("Number of Product:",len(Amazon_Sales['product_id'].unique()))
print("Number of User who gave review:",len(Amazon_Sales['user_id'].unique()))

Number of Rating: 1464
Number of Product: 1350
Number of User who gave review: 1193


**Approach for this objective: Collaborative Filtering**

Collaborative filtering is a popular technique used in recommender systems to generate personalized recommendations for users based on their past behavior or preferences, as well as the behavior of similar users. It relies on the idea that users who have agreed in the past tend to agree again in the future. Collaborative filtering methods do not require explicit knowledge about the items being recommended, making them particularly useful in scenarios where item characteristics are unknown or difficult to model.

In this particular objective, User-based collaborative filtering is used. This approach recommends items to a user by finding other users who are similar to that user based on their past interactions with items. The assumption is that if two users have similar preferences in the past, they are likely to have similar preferences in the future. Once similar users are identified, items that they have liked or interacted with, but the target user has not, can be recommended to the target user

In [14]:
# Now, we create user-item matrix using scipy csr matrix
from scipy.sparse import csr_matrix

def create_matrix(df):

	N = len(df['user_id'].unique())
	M = len(df['product_id'].unique())

	# Map Ids to indices
	user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
	product_mapper = dict(zip(np.unique(df["product_id"]), list(range(M))))

	# Map indices to IDs
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
	product_inv_mapper = dict(zip(list(range(M)), np.unique(df["product_id"])))

	user_index = [user_mapper[i] for i in df['user_id']]
	product_index = [product_mapper[i] for i in df['product_id']]

	X = csr_matrix((df["rating"], (product_index, user_index)), shape=(M, N))

	return X, user_mapper, product_mapper, user_inv_mapper, product_inv_mapper

X, user_mapper, product_mapper, user_inv_mapper, product_inv_mapper = create_matrix(Amazon_Sales)



Implementing Nearest Neighbour

In [15]:
from sklearn.neighbors import NearestNeighbors
def find_similar_product(product_id, X, k, metric='cosine', show_distance=False):

	neighbour_ids = []

	product_ind = product_mapper[product_id]
	product_vec = X[product_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	product_vec = product_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(product_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(product_inv_mapper[n])
	neighbour_ids.pop(0)
	return neighbour_ids

Product Recommendation when you buy/choose a product


In [26]:
product_titles = dict(zip(Amazon_Sales['product_id'], Amazon_Sales['product_name']))

product_id = "B098NS6PVG"

similar_ids = find_similar_product(product_id, X, k=10)
product_title = product_titles[product_id]

print(f"Since you bought {product_title}")
print("You might want to buy: ")
for i in similar_ids:
    print(product_titles[i])
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

Since you bought Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)
You might want to buy: 
Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Micro USB Cable for Smartphones, Tablets, Laptops & Other Micro USB Devices, 480Mbps Data Sync, Quick Charge 3.0 (RCM15, Black)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C to Type C Cable for Smartphones, Tablets, Laptops & Other Type C Devices, PD Technology, 480Mbps Data Sync (RCTT15, Black)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Glen 3 in 1 Elect

Product Recommendation for user

In [17]:
def recommend_products_for_user(user_id, X, user_mapper, product_mapper, product_inv_mapper, k=10):
  df1 = Amazon_Sales[Amazon_Sales['user_id'] == user_id]

  if df1.empty:
    print(f"User with ID {user_id} does not exist.")
    return

  product_id = df1[df1['rating'] == max(df1['rating'])]['product_id'].iloc[0]

  product_titles = dict(zip(Amazon_Sales['product_id'], Amazon_Sales['product_name']))

  similar_ids = find_similar_product(product_id, X, k)
  product_title = product_titles.get(product_id, "Movie not found")

  if product_title == "Movie not found":
    print(f"Movie with ID {product_id} not found.")
    return

  print(f"Since you bought {product_title}")
  print("You might also want to buy: ")
  for i in similar_ids:
    print(product_titles.get(i, "Movie not found"))
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")


In [28]:
user_id = "AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZHM4QB2KPKFUQ,AFK4NJOLFSJGWLOJIUIAROJF6YVA,AFUOTYRFUXVPEBGIXVZZ7DR3CZUA,AFDLRSXKDZ6U3U3KD46SQLFGZQRA,AH5VLM66SIK7J3IRG4NY7XVOQ55A,AE3MQNNHHLUHXURL5S7IAR7JTGNQ,AFSEOFZY67MYC7UAJU264Z5NFTLA"
recommend_products_for_user(user_id, X, user_mapper, product_mapper, product_inv_mapper, k=10)

Since you bought Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)
You might also want to buy: 
Duracell 38W Fast Car Charger Adapter with Dual Output. Quick Charge, Type C PD 20W & Qualcomm Certified 3.0 Compatible for iPhone, All Smartphones, Tablets & More (Copper & Black)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Croma 80 cm (32 Inches) HD Ready LED TV (CREL7369, Black) (2021 Model)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
HP 150 Wireless USB Mouse with Ergonomic and ambidextrous Design, 1600 DPI Optical Tracking, 2.4 GHz Wireless connectivity, Dual-Function Scroll Wheel and 12 Month Long Battery Life. 3-Years Warranty.
------------